In [38]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

In [39]:
df1 = pd.read_csv('last12months/12months/204300479.csv')
df2 = pd.read_csv('last12months/12months/220330932.csv')
df3 = pd.read_csv('last12months/12months/224228487.csv')

/Users/leonardozecchin/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (21,23,25,27,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/leonardozecchin/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (24,26,49,50,51,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [40]:
df1['dt'] = pd.to_datetime(df1['TIMESTAMP'],unit='ms')
df2['dt'] = pd.to_datetime(df2['TIMESTAMP'],unit='ms')
df3['dt'] = pd.to_datetime(df3['TIMESTAMP'],unit='ms')

In [41]:
def cleanCSV(df):
  cols_to_check = df.columns
  arr_col = []
  for col in df.columns:
    if df[col].isna().all():
        print("La colonna", col, "contiene solo valori NaN.")
        arr_col.append(col)
  df_cleaned = df.drop(columns=arr_col)
  df_cleaned = df_cleaned.replace('---', 0)
  return df_cleaned

In [42]:
def sortDf(df):
    if 'TIMESTAMP' in df.columns:
        df = df.sort_values('TIMESTAMP',ascending=True)
        df.index = range(len(df))
    else:
        print("La colonna TIMESTAMP non è presente nel dataframe.")
        return None
    return df

In [43]:
def resampleDf(df,datetime):
    if 'dt' in df.columns:
        df.set_index('dt', inplace=True)
    df_noNan = df.ffill()
    df_resampled = df_noNan.resample(datetime).ffill()
    df_resampled = df_resampled.replace('---', 0)
    return df_resampled
    

In [44]:
def checkFileInFolder(folder, file):
    if file in os.listdir(folder):
        return True
    return False

In [45]:
def getResampledDf(df,file_path):
    if not checkFileInFolder(folder ='last12months/12months',file=file_path):
        df_sorted = sortDf(df)
        df_sorted = cleanCSV(df_sorted)
        df_resampled = resampleDf(df_sorted,'30s')
        df_resampled.to_csv('last12months/12months/' + file_path)
        return df_sorted, df_resampled
    else:
        df_resampled = pd.read_csv('last12months/12months/'+ file_path)
        return None, df_resampled

    

In [9]:
df1_sorted, df1_resampled = getResampledDf(df1,'204300479_resampled30s.csv')
df2_sorted, df2_resampled = getResampledDf(df2,'220330932_resampled30s.csv')
df3_sorted, df3_resampled = getResampledDf(df3,'224228487_resampled30s.csv')

La colonna Cloud Status contiene solo valori NaN.
La colonna Modo Impianto contiene solo valori NaN.
La colonna Modo Impianto contiene solo valori NaN.
La colonna Modo Impianto contiene solo valori NaN.


In [10]:
def getUniques(df):
    cols = df.columns
    dict_unique = dict.fromkeys(cols,None)
    for col in cols:
        dict_unique[col] = df[col].unique()
    return dict_unique

In [46]:
def getBooelan(dict):
    arr = []
    for key, value in dict.items():
        if len(value) < 4:
            b = True
            for v in value:
                # print(v,pd.isna(v))
                if pd.isna(v)==False:
                    if int(v) != 0 and int(v) != 1:
                        # print('ciao')
                        b = False
                        break
            # print(value,b)
            if b:
                arr.append(key)
    return arr

In [47]:
def printUniques(dict,tp):
    booleans = getBooelan(dict)
    for key, value in dict.items():
        counts = []
        if key in booleans and tp ==1:
            print(key, value)
        elif tp==2 and len(value) > 4 :
            print(key, value[0:5])

In [48]:
def countOccurences(df,dict):
    booleans = getBooelan(dict)
    counters = dict.fromkeys(booleans,None)
    for key, value in dict.items():
        if key in booleans:
            value = value[1:]
            counts = dict.fromkeys(value,None)
            for v in value:
                if not pd.isna(v):
                    counts[v]= df[key].value_counts()[v]
            counters[key] = counts
            # print(key, counts, max(counts))
    return counters
            

In [49]:
def getMax(dict):
    maxs = dict.fromkeys(dict.keys(),None)
    for key, value in dict.items():
        # print(key, value)
        for k, v in value.items():
            if v == max(value.values()):
                maxs[key] = k
    return maxs

In [50]:
def plotColumn(df,col):
    if col not in df.columns:
        print("La colonna", col, "non è presente nel dataframe.")
        return None
    df_toPlot = df[col]
    plt.plot(df_toPlot)
    try: 
        plt.ylim(0, max(df_toPlot.unique())+1)
    except:
        pass

    plt.show()


In [51]:
dict_unique1 = getUniques(df1_resampled)
dict_unique2 = getUniques(df2_resampled)
dict_unique3 = getUniques(df3_resampled)
arr_booleani1 = getBooelan(dict_unique1)
arr_booleani2 = getBooelan(dict_unique2)
arr_booleani3 = getBooelan(dict_unique3)


## Fill Nan with the values of another csv

In [17]:
if 'last_20Row_204300479.csv' in os.listdir('last12months/12months'):
    df1_last20 = pd.read_csv('last12months/12months/last_20Row_204300479.csv')
if 'd_test_filled.csv' in os.listdir('last12months/12months'):
    d_test_filled = pd.read_csv('last12months/12months/d_test_filled.csv')

In [18]:
def getLastValues(df):
    d_sample = df.iloc[-1]
    dictionary = dict.fromkeys(df.columns,None)

    for col in df.columns:
        if col in d_sample.index:
            dictionary[col] = d_sample[col]

    return dictionary

In [19]:
def reversedMax(dizionario):
    for key, value in dizionario.items():
        if value == 1:
            dizionario[key] = 0
        else:
            dizionario[key] = 1
    return dizionario

In [20]:
def getGeneralBooleans(df1,df2,df3):
    b1 = getBooelan(getUniques(df1))
    b2 = getBooelan(getUniques(df2))
    b3 = getBooelan(getUniques(df3))
    intersection = list(set(b1) & set(b2) & set(b3))
    intersection.sort()
    return intersection

In [21]:
def fillBooleans(df,booleans,dizionario):
    for key in booleans:
        if key != 'Modalità Estate/Inverno (solo scrittura)' and key in dizionario.keys():
            print(f'{key}: {dizionario[key]}')        
            df[key] = df[key].replace(np.nan,dizionario[key])
    return df

In [22]:
def fillOthers(df,sample):
    col_s = sample.index
    booleani = getBooelan(getUniques(df))
    for col in df.columns:
        # print(col)
        if col == 'Anomalia #1':
            df[col] = df[col].replace(np.nan,sample[col])
        if col not in booleani and col in col_s:
            # print('in')
            df[col] = df[col].replace(np.nan,sample[col])
    return df

In [23]:
d1 = pd.read_csv('last12months/20230721-153054/204300479_LUNA IN PLUS AIR.csv')
d2 = pd.read_csv('last12months/20230721-153054/220330932_LUNA IN PLUS AIR.csv')
d3 = pd.read_csv('last12months/20230721-153054/224228487_LUNA IN PLUS AIR.csv')

In [24]:
d1 = cleanCSV(d1)
d1['dt'] = pd.to_datetime(d1['TIMESTAMP'],unit='ms')
d1_sorted = sortDf(d1)
d1_resampled = resampleDf(d1_sorted,'30s')
d1_resampled = d1_resampled.replace('---', 0)

La colonna Cloud Status contiene solo valori NaN.
La colonna Modo Impianto contiene solo valori NaN.
La colonna Modo service contiene solo valori NaN.
La colonna Taratura sensore contiene solo valori NaN.


In [25]:
d1_resampled.tail()

,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,Energia totale globale,Energia totale globale delta,...,Standby giornaliero,Stato attivazione BOOST sanitario,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
dt,,,,,,,,,,,,,,,,,,,,,
2022-06-30 21:57:30,1656626247976,1.0,0.0,3486319.0,1335423.0,1.0,145.0,90.0,235.0,0.0,...,0.0,0.0,43.2,0.0,0.0,0.0,41.9,0.0,0.0,-63.0
2022-06-30 21:58:00,1656626277879,1.0,0.0,3486319.0,1335423.0,1.0,145.0,90.0,235.0,0.0,...,0.0,0.0,42.9,0.0,0.0,0.0,41.5,0.0,0.0,-63.0
2022-06-30 21:58:30,1656626307641,1.0,0.0,3486319.0,1335423.0,1.0,145.0,90.0,235.0,0.0,...,0.0,0.0,42.9,0.0,0.0,0.0,41.5,0.0,0.0,-63.0
2022-06-30 21:59:00,1656626328421,1.0,0.0,3486319.0,1335423.0,1.0,145.0,90.0,235.0,0.0,...,0.0,0.0,42.7,0.0,0.0,0.0,41.2,0.0,0.0,-62.0
2022-06-30 21:59:30,1656626368642,1.0,0.0,3486319.0,1335423.0,1.0,145.0,90.0,235.0,0.0,...,0.0,0.0,42.6,0.0,0.0,0.0,41.1,0.0,0.0,-62.0


In [26]:
if 'd1.csv' not in os.listdir('last12months'):
    d1_resampled.tail(50).to_csv('last12months/d1.csv')
if 'df1.csv' not in os.listdir('last12months'):
    df1_resampled.head(50).to_csv('last12months/df1.csv')

In [27]:
last_rows_d1= d1_resampled.tail(5)
dizionario = getLastValues(last_rows_d1)
dizionario

{'TIMESTAMP': 1656626368642,
 'Abilitazione al conteggio energia': 1.0,
 'Anomalia #1': 0.0,
 'Caldaia': 3486319.0,
 'Caldaia giornaliero': 1335423.0,
 'Connection Status': 1.0,
 'Energia parziale in riscaldamento caldaia': 145.0,
 'Energia parziale in sanitario caldaia': 90.0,
 'Energia totale globale': 235.0,
 'Energia totale globale delta': 0.0,
 'Energia totale globale per day': 2.0,
 'Energia totale globale per month': 56.0,
 'Energia totale globale per week': 7.0,
 'Energia totale in riscaldamento caldaia': 145.0,
 'Energia totale in sanitario caldaia': 90.0,
 'Fattore di correzione': 1.0,
 'Flame status': 0.0,
 'Flusso sanitario': 0,
 'Intercetta della retta della potenza': -1.22,
 'Livello modulazione': 0.0,
 'M64A17E8': 91.0,
 'M64A17E9': 0.0,
 'M64A17EA': 91.0,
 'M64A17EB': 0.0,
 'M64A17EC': '005A',
 'M64A17ED': 0.0,
 'M64A17EE': '005A',
 'M64A17EF': 0.0,
 'Media T esterna': 12.0,
 'Modalità Estate/Inverno (solo scrittura)': 1.0,
 'Modo vacanza': 0.0,
 'Pendenza della retta d

In [ ]:
generalBooleans = getGeneralBooleans(df1_resampled,df2_resampled,df3_resampled)
d_test_fB = fillBooleans(df1_resampled,generalBooleans,dizionario)
d_campione = d1_resampled.iloc[-1]
d_test_filled = fillOthers(d_test_fB,d_campione)
d_test_filled = cleanCSV(d_test_filled)

In [ ]:
d_test_filled.head()

In [ ]:
first10 = d_test_filled.head(10)
first10.to_excel('last12months/12months/first_10Row_204300479.xlsx')

In [ ]:
d1_resampled.tail()

In [ ]:
d_test_fB.head()

In [ ]:
if 'd_test_filled.csv' not in os.listdir('last12months/12months'):
    d_test_filled.to_csv('last12months/12months/d_test_filled.csv')

In [ ]:

if 'last_20Row_204300479.csv' not in os.listdir('last12months/12months'):
    d_finale20 = d_test.head(20)
    d_finale20.to_csv('last12months/12months/last_20Row_204300479.csv') 

In [ ]:
if 'last_20Row_204300479.xlsx' not in os.listdir('last12months/12months'):
    df1_last20.to_excel('last12months/12months/last_20Row_204300479.xlsx')

In [ ]:
d_test_filled.head()

In [34]:
dt = pd.read_csv('last12months/12months/d_test_filled.csv')
dt.shape

/Users/leonardozecchin/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (22,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1084089, 57)

In [33]:
df1_resampled.shape

(1084089, 55)

In [36]:
dt.tail()

,Unnamed: 0,dt,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
1084084,1084084,2023-07-12 08:02:00,1689148915763,1.0,0.0,3368569.0,417542.0,1.0,121.0,231.0,...,0.0,-2.3,34.7,0.0,0.0,0.0,31.5,0.0,0.0,-49.0
1084085,1084085,2023-07-12 08:02:30,1689148945527,1.0,0.0,3368569.0,417542.0,1.0,121.0,231.0,...,0.0,-2.3,34.7,0.0,0.0,0.0,31.5,0.0,0.0,-49.0
1084086,1084086,2023-07-12 08:03:00,1689148976831,1.0,0.0,3368569.0,417542.0,1.0,121.0,231.0,...,0.0,-2.3,34.6,0.0,0.0,0.0,31.4,0.0,0.0,-49.0
1084087,1084087,2023-07-12 08:03:30,1689149006629,1.0,0.0,3368569.0,417542.0,1.0,121.0,231.0,...,0.0,-2.3,34.6,0.0,0.0,0.0,31.4,0.0,0.0,-49.0
1084088,1084088,2023-07-12 08:04:00,1689149036525,1.0,0.0,3368569.0,417542.0,1.0,121.0,231.0,...,0.0,-2.3,34.5,0.0,0.0,0.0,31.4,0.0,0.0,-49.0


In [37]:
dt.head()

,Unnamed: 0,dt,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
0,0,2022-06-30 22:00:00,1656626400000,1.0,0.0,3486319.0,739466.0,1.0,145.0,90.0,...,0.0,NaN,42.6,0.0,0.0,0.0,41.1,0.0,0.0,-62.0
1,1,2022-06-30 22:00:30,1656626428307,1.0,0.0,3486319.0,739466.0,1.0,145.0,90.0,...,0.0,NaN,42.6,0.0,0.0,0.0,41.1,0.0,0.0,-62.0
2,2,2022-06-30 22:01:00,1656626449120,1.0,0.0,3486319.0,739466.0,1.0,145.0,90.0,...,0.0,NaN,42.1,0.0,0.0,0.0,40.4,0.0,0.0,-66.0
3,3,2022-06-30 22:01:30,1656626489432,1.0,0.0,3486319.0,739466.0,1.0,145.0,90.0,...,0.0,NaN,42.1,0.0,0.0,0.0,40.1,0.0,0.0,-60.0
4,4,2022-06-30 22:02:00,1656626519228,1.0,0.0,3486319.0,739466.0,1.0,145.0,90.0,...,0.0,NaN,41.7,0.0,0.0,0.0,39.8,0.0,0.0,-60.0


In [53]:
print("Anomalies' values: ",df1_resampled['Anomalia #1'].unique())
print("Anomalies' values: ",df2_resampled['Anomalia #1'].unique())
print("Anomalies' values: ",df3_resampled['Anomalia #1'].unique())


Anomalies' values:  [nan  0.]
Anomalies' values:  [nan  0. 76.]
Anomalies' values:  [nan  0. 85.]


In [60]:
num_anomaliesDf2 = df2['Anomalia #1'].value_counts()[76]
num_anomaliesDf3 = df3['Anomalia #1'].value_counts()[85]
print("Number of anomalies type 76 in df2:", num_anomaliesDf2)
print("Number of anomalies type 85 in df3:", num_anomaliesDf3)

Number of anomalies type 76 in df2: 9
Number of anomalies type 85 in df3: 1


9